<a href="https://colab.research.google.com/github/Satyake/NLP/blob/master/Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense,Bidirectional
import numpy as np 

batch_size=64
epochs=10
latent_dim=256 #latent dim of the encoding space
num_samples=10000
path='/content/eng-fra.txt'

In [76]:
#vectorize
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(path ,'r',encoding='utf8') as file:
  lines=file.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
  input_text,target_text=line.split('\t') 
  #Tab for starting the sequence
  #\n as the end sequence
  target_text='\t'+target_text+'\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
      target_characters.add(char)


In [78]:
input_texts

['Go.',
 'Run!',
 'Run!',
 'Wow!',
 'Fire!',
 'Help!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'Oh no!',
 'Attack!',
 'Attack!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Get up.',
 'Got it!',
 'Got it!',
 'Got it?',
 'Got it?',
 'Got it?',
 'Hop in.',
 'Hop in.',
 'Hug me.',
 'Hug me.',
 'I fell.',
 'I fell.',
 'I know.',
 'I left.',
 'I left.',
 'I lost.',
 "I'm 19.",
 "I'm OK.",
 "I'm OK.",
 'Listen.',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'Really?',
 'Really?',
 'Really?',
 'Thanks.',
 'We try.',
 'We won.',
 'We won.',
 'We won.',
 'We won.',
 'Ask Tom.',
 'Awesome!',
 'Be calm.',
 'Be calm.',
 'Be calm.',
 'Be cool.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be kind.',
 'Be nice.',
 'Be nice.',
 'Be nice.',
 'Be nice.',
 'Be nice.',
 'Be nice.',
 'Beat it.',
 'Call me.',
 'Call me.',
 'Call us.',
 'Call us.',
 'Come in.',
 'Come in.',


In [79]:
target_texts

['\tVa !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tÇa alors\u202f!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tJe comprends.\n',
 "\tJ'essaye.\n",
 "\tJ'ai gagné !\n",
 "\tJe l'ai emporté !\n",
 '\tOh non !\n',
 '\tAttaque !\n',
 '\tAttaquez !\n',
 '\tSanté !\n',
 '\tÀ votre santé !\n',
 '\tMerci !\n',
 '\tLève-toi.\n',
 "\tJ'ai pigé !\n",
 '\tCompris !\n',
 '\tPigé\u202f?\n',
 '\tCompris\u202f?\n',
 "\tT'as capté\u202f?\n",
 '\tMonte.\n',
 '\tMontez.\n',
 '\tSerre-moi dans tes bras !\n',
 '\tSerrez-moi dans vos bras !\n',
 '\tJe suis tombée.\n',
 '\tJe suis tombé.\n',
 '\tJe sais.\n',
 '\tJe suis parti.\n',
 '\tJe suis partie.\n',
 "\tJ'ai perdu.\n",
 "\tJ'ai 19 ans.\n",
 '\tJe vais bien.\n',
 '\tÇa va.\n',
 '\tÉcoutez !\n',
 '\tImpossible\u202f!\n',
 '\tEn aucun cas.\n',
 "\tC'est hors de question !\n",
 "\tIl n'en est pas question !\n",
 "\tC'est exclu 

In [80]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '5',
 '6',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f'}

In [81]:
input_characters

{' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '’'}

In [82]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(txt)for txt in input_texts])
max_decoder_seq_length=max([len(txt)for txt in target_texts])


In [83]:
print('number of samples',len(input_texts))
print('Number of unique input tokens',num_encoder_tokens)
print('number of unique output tokens',num_decoder_tokens)
print('Maximum Sentence_english_length',max_encoder_seq_length)
print('Maximum Sentence_french_length',max_decoder_seq_length)

number of samples 10000
Number of unique input tokens 71
number of unique output tokens 91
Maximum Sentence_english_length 16
Maximum Sentence_french_length 59


In [84]:
input_token_index=dict([(char,i)for i ,char in enumerate(input_characters)])
target_token_index=dict([(char,i)for i ,char in enumerate(target_characters)])


In [ ]:
input_token_index

In [ ]:
#encoder input to take shape (num_pairs,max_english_sentence_length,num_english_characters)
#decoder target to take shape (num_pairs,max_french_sentence_length,num_french_characters)
#one hot vectorizations
#keras documentation 

In [86]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')

In [ ]:
encoder_input_data

In [87]:
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [88]:
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [89]:
#One Hot Repr
for i ,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t ,char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]]=1
  encoder_input_data[i,t+1:,input_token_index[' ']]=1
  for t,char in enumerate(target_text):
    #decoder target_data is ahead of decoder_input_data by 1 time step
    decoder_input_data[i,t,target_token_index[char]]=1
    if t>0:
      decoder_target_data[i,t-1,target_token_index[char]]=1
    decoder_input_data[i,t+1:,target_token_index[' ']]=1
    decoder_target_data[i,t:,target_token_index[' ']]=1
    
                     


In [98]:
encoder_input_data[15]

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [99]:
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]

In [100]:
decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [ ]:
model=Model(inputs=[encoder_inputs,decoder_inputs],outputs=decoder_outputs)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.4)

Epoch 1/10
94/94 [==============================] - 42s 449ms/step - loss: 1.6675 - accuracy: 0.9723 - val_loss: 1.4816 - val_accuracy: 0.9831
Epoch 2/10
94/94 [==============================] - 41s 440ms/step - loss: 1.2338 - accuracy: 0.9831 - val_loss: 1.4116 - val_accuracy: 0.9831
Epoch 3/10
94/94 [==============================] - 41s 440ms/step - loss: 1.1527 - accuracy: 0.9834 - val_loss: 1.2893 - val_accuracy: 0.9839
Epoch 4/10
94/94 [==============================] - 42s 442ms/step - loss: 1.0715 - accuracy: 0.9893 - val_loss: 1.2204 - val_accuracy: 0.9914
Epoch 5/10
94/94 [==============================] - 41s 441ms/step - loss: 1.0178 - accuracy: 0.9921 - val_loss: 1.1744 - val_accuracy: 0.9851
Epoch 6/10
94/94 [==============================] - 42s 442ms/step - loss: 0.9820 - accuracy: 0.9902 - val_loss: 1.1421 - val_accuracy: 0.9941
Epoch 7/10
94/94 [==============================] - 41s 438ms/step - loss: 0.9511 - accuracy: 0.9879 - val_loss: 1.1104 - val_accuracy: 0.9890